## Web Scraping Automatizado: ScienceDirect

Este notebook implementa un **bot de web scraping automatizado** para descargar artículos científicos en formato BibTeX desde ScienceDirect.

### Objetivos:
1. **Autenticación automática** vía proxy institucional
2. **Navegación programática** a ScienceDirect
3. **Búsqueda automatizada** de términos específicos
4. **Descarga masiva** de archivos BibTeX con paginación
5. **Organización** de archivos descargados

### Flujo del Proceso:
```
Login Proxy → Autenticación Google → Selección Facultad → 
Acceso ScienceDirect → Búsqueda → Configuración Resultados → 
Loop de Paginación → Descarga BibTeX → Renombrado → Siguiente Página
```

## Paso 1 — Imports y carga de variables de entorno

Este bloque importa las librerías necesarias para manejar datos, controlar el navegador con Selenium y cargar variables desde un archivo .env.

In [ ]:
#imports requeridos para que el proyecto funcione
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import os
from dotenv import load_dotenv

# Cargar variables del archivo .env
load_dotenv()

# Obtener y validar variables de entorno
email = os.getenv("EMAIL")
password = os.getenv("PASSWORD")

# Validación crítica de variables
if not email or not password:
    raise ValueError("ERROR: EMAIL y PASSWORD deben estar definidos en el archivo .env")

print(f"Variables cargadas correctamente")
print(f"Email: {email[:3]}***@{email.split('@')[1] if '@' in email else '***'}")

## Paso 2 — Configuración de ruta y opciones de descarga en Chrome

Este bloque define la carpeta donde se guardarán las descargas y configura las preferencias del navegador Chrome para automatizar ese proceso.

In [ ]:
# Configurar la ruta de descarga
download_path = os.getenv("DOWNLOAD_PATH")

# Validación crítica de ruta de descarga
if not download_path:
    raise ValueError("ERROR: DOWNLOAD_PATH debe estar definido en el archivo .env")

os.makedirs(download_path, exist_ok=True)

chrome_options = webdriver.ChromeOptions()

# Configuración de preferencias de descargas
prefs = {
    "download.default_directory": download_path,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False,
    "profile.default_content_settings.popups": 0,
    "profile.content_settings.exceptions.automatic_downloads.*.setting": 1
}
chrome_options.add_experimental_option("prefs", prefs)

# Argumentos adicionales para estabilidad
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-download-notification")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

print(f"Opciones de Chrome configuradas")
print(f"Ruta de descarga: {download_path}")

## Paso 3 — Inicialización del navegador y apertura de la página

Este bloque crea una instancia del navegador Chrome con las opciones configuradas previamente y accede a la URL deseada.

In [ ]:
# Iniciar el navegador con protección
driver = None

try:
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(30)
    print("Navegador Chrome iniciado correctamente")
    
    # Abrir la URL
    url1 = 'https://login.intelproxy.com/v2/inicio?cuenta=7Ah6RNpGWF22jjyq'
    driver.get(url1)
    print(f"Navegando a: {url1}")
    
except Exception as e:
    print(f"ERROR: Error al iniciar el navegador: {e}")
    if driver:
        driver.quit()
    raise

## Paso 4 — Espera inicial para carga de página

Este bloque realiza una pausa para asegurar que la página se cargue completamente antes de continuar.

In [4]:
time.sleep(5)

## Paso 5 — Esperar y hacer clic en el botón de inicio de sesión

Este bloque espera hasta que el botón de inicio de sesión esté disponible y realiza el clic para continuar con el proceso.

In [5]:
login_button2 = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/div/div[1]/div[2]/a")))
login_button2.click()

## Paso 6 — Espera para transición de página

Este bloque realiza una pausa para permitir la transición a la página de autenticación.

In [6]:
time.sleep(5)

## Paso 7 — Ingresar el correo electrónico

Este bloque espera el campo de correo, introduce el valor obtenido del archivo .env y envía la tecla Enter para continuar.

In [7]:
email_input = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "identifierId"))
)

email_input.send_keys(email)
email_input.send_keys(Keys.RETURN)

## Paso 8 — Espera para validación de correo

Este bloque realiza una pausa para permitir la validación del correo electrónico.

In [8]:
time.sleep(3)

## Paso 9 — Ingresar la contraseña

Este bloque espera el campo de contraseña, escribe la clave obtenida del archivo .env y envía la tecla Enter para iniciar sesión.

In [9]:
password_input = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.NAME, "Passwd"))
)# Nombre del campo de contraseña
password_input.send_keys(password)
password_input.send_keys(Keys.RETURN)

## Paso 10 — Espera para autenticación

Este bloque realiza una pausa para permitir que se complete el proceso de autenticación.

In [10]:
time.sleep(3)

## Paso 11 — Acceder al apartado "Fac. Ingeniería"

Este bloque localiza y hace clic en el botón que despliega la sección correspondiente a "Fac. Ingeniería".

In [11]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Buscar el botón <summary> que contiene "Fac. Ingeniería"
boton = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, "//summary[contains(., 'Fac. Ingeniería')]"))
)

time.sleep(2)
boton.click()

## Paso 12 — Espera para carga de opciones

Este bloque realiza una pausa para permitir que se carguen las opciones de la facultad.

In [12]:
time.sleep(2)

## Paso 13 — Localizar el enlace a ScienceDirect

Este bloque espera a que el enlace correspondiente a "ScienceDirect" esté disponible para su interacción en la página.

In [13]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Buscar el enlace a IEEE por su XPath correcto
enlaceSD = WebDriverWait(driver, 15).until(
    EC.element_to_be_clickable((By.XPATH, "//*[@id='facingenierasciencedirectdescubridor']//h2[@class='result-title']/a"))
)




## Paso 14 — Hacer clic en el enlace de ScienceDirect

Este bloque realiza la acción de clic en el enlace previamente localizado para acceder al contenido o recurso correspondiente.

In [14]:
# Hacer clic en el enlace
enlaceSD.click()

## Paso 15 — Activar la barra de búsqueda en la página de ScienceDirect

Este bloque espera a que la barra de búsqueda esté disponible y escribe el término deseado para realizar la consulta.

In [15]:
# Esperar a que la barra de búsqueda esté en el DOM
barra_busqueda = WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div/div/div[1]/div[2]/div[2]/div/div/form/div[1]/div[1]/div[1]/input"))
)

# Escribir "generative artificial intelligence"
barra_busqueda.send_keys('generative artificial intelligence')

## Paso 16 — Ejecutar la búsqueda

Este bloque localiza el botón de búsqueda y realiza el clic para iniciar la consulta en la página.

In [16]:
# Esperar a que el botón de búsqueda esté clickeable
boton_buscar = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/div[1]/div[2]/div[2]/div/div/form/div[2]/button"))
)

boton_buscar.click()

## Paso 17 — Espera para carga de resultados

Este bloque realiza una pausa para permitir que se carguen los resultados de la búsqueda.

In [17]:
time.sleep(5)

## Paso 18 — Verificar la finalización de la descarga

Este bloque define una función que espera la aparición de un nuevo archivo .bib en la carpeta de descargas, confirmando que la descarga se completó correctamente.

In [18]:
# Función para verificar que la descarga se completó
def esperar_descarga(carpeta, tiempo_max=60):
    """
    Espera hasta que aparezca un nuevo archivo .bib en la carpeta de descargas
    o hasta que se agote el tiempo máximo.
    """
    inicio = time.time()
    archivos_iniciales = set([f for f in os.listdir(carpeta) if f.endswith('.bib')])
    
    while time.time() - inicio < tiempo_max:
        time.sleep(2)  # Comprobar cada 2 segundos
        archivos_actuales = set([f for f in os.listdir(carpeta) if f.endswith('.bib')])
        nuevos_archivos = archivos_actuales - archivos_iniciales
        
        if nuevos_archivos:
            # Encontró nuevo archivo
            nuevo_archivo = list(nuevos_archivos)[0]
            return os.path.join(carpeta, nuevo_archivo)
        
    print("⚠️ Tiempo de espera de descarga agotado.")
    return None

## Paso 19 — Espera adicional antes de configuración

Este bloque realiza una pausa adicional antes de configurar las opciones de paginación.

In [19]:
time.sleep(3)

## Paso 20 — Mostrar más resultados por página

Este bloque aumenta la cantidad de resultados visibles en la página a 100, facilitando el acceso y descarga de más elementos sin cambiar de página.

In [20]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

try:
    # Esperar a que el menú de paginación esté presente
    wait = WebDriverWait(driver, 30)
    
    # Primero localizamos el contenedor de la paginación
    pagination_container = wait.until(
        EC.presence_of_element_located((By.ID, "srp-pagination-options"))
    )
    
    # Ahora buscamos específicamente el enlace para 100 resultados
    # Usando el atributo data-aa-name que es único para este botón
    cienItemsPerPage_link = wait.until(
        EC.element_to_be_clickable((
            By.CSS_SELECTOR, 
            'a[data-aa-name="srp-100-results-per-page"]'
        ))
    )
    
    # Hacer scroll hasta el elemento
    driver.execute_script("arguments[0].scrollIntoView(true);", cienItemsPerPage_link)
    time.sleep(1)  # Pequeña pausa para asegurar que el scroll se complete
    
    # Hacer clic usando JavaScript para evitar problemas de superposición
    driver.execute_script("arguments[0].click();", cienItemsPerPage_link)
    
    # Esperar a que la página se actualice
    time.sleep(3)
    
    print("Se hizo clic exitosamente en '100 resultados por página'")
    
except Exception as e:
    print(f"Ocurrió un error: {str(e)}")

Se hizo clic exitosamente en '100 resultados por página'


## Paso 21 — Espera para aplicar configuración

Este bloque realiza una pausa para permitir que se aplique la configuración de 100 resultados por página.

In [21]:
time.sleep(5)

## Paso 22 — Descarga automatizada de resultados en formato .bib

Este bloque implementa la automatización completa para descargar los resultados de búsqueda de ScienceDirect en formato BibTeX, página por página, guardándolos con nombres personalizados. Incluye:

- **Detección automática del número total de páginas**
- **Selección de todos los artículos** en cada página
- **Exportación en formato BibTeX**
- **Reintentos automáticos** en caso de fallo
- **Navegación automática** entre páginas
- **Organización de archivos** en carpeta dedicada

In [22]:
# Código para reemplazar en tu notebook Jupyter
# Copia y pega este código en una nueva celda

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import re


def esperar_descarga(download_path, archivos_antes, timeout=30):
    """Espera a que aparezca un nuevo archivo .bib en la carpeta de descargas"""
    end_time = time.time() + timeout
    while time.time() < end_time:
        archivos_actuales = set(f for f in os.listdir(download_path) if f.endswith(".bib"))
        nuevos = archivos_actuales - archivos_antes
        if nuevos:
            nuevo_archivo = max([os.path.join(download_path, f) for f in nuevos],
                              key=os.path.getctime)
            return nuevo_archivo
        time.sleep(1)
    return None


def obtener_total_paginas(driver):
    """
    Detecta automáticamente el número total de páginas disponibles
    """
    try:
        # Buscar el texto que indica el total de resultados
        # Ejemplo: "1-100 of 1,234 results"
        results_info = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".results-info"))
        )
        
        texto_resultados = results_info.text
        print(f"📊 Información de resultados: {texto_resultados}")
        
        # Extraer el número total de resultados usando regex
        # Buscar patrones como "of 1,234 results" o "of 1234 results"
        match = re.search(r'of\s+([\d,]+)\s+results', texto_resultados)
        if match:
            total_resultados = int(match.group(1).replace(',', ''))
            print(f"📈 Total de resultados encontrados: {total_resultados}")
            
            # Calcular número de páginas (asumiendo 100 resultados por página)
            resultados_por_pagina = 100
            total_paginas = (total_resultados + resultados_por_pagina - 1) // resultados_por_pagina
            
            print(f"📄 Total de páginas calculadas: {total_paginas}")
            return total_paginas
        
        # Método alternativo: buscar directamente en la paginación
        try:
            pagination_links = driver.find_elements(By.CSS_SELECTOR, ".pagination a")
            if pagination_links:
                # Buscar el número más alto en los enlaces de paginación
                numeros_pagina = []
                for link in pagination_links:
                    texto = link.text.strip()
                    if texto.isdigit():
                        numeros_pagina.append(int(texto))
                
                if numeros_pagina:
                    max_pagina = max(numeros_pagina)
                    print(f"📄 Páginas detectadas por paginación: {max_pagina}")
                    return max_pagina
        except:
            pass
            
    except Exception as e:
        print(f"⚠️ No se pudo detectar el total de páginas automáticamente: {e}")
    
    # Si no se puede detectar, usar un valor por defecto alto
    print("🔄 Usando detección dinámica página por página...")
    return None


def verificar_pagina_siguiente_existe(driver):
    """
    Verifica si existe un botón de 'siguiente página'
    """
    try:
        boton_siguiente = driver.find_element(By.XPATH, "//a[@data-aa-name='srp-next-page']")
        # Verificar si el botón está habilitado (no deshabilitado)
        clases = boton_siguiente.get_attribute("class") or ""
        aria_disabled = boton_siguiente.get_attribute("aria-disabled")
        
        # Si tiene clase 'disabled' o aria-disabled='true', no hay más páginas
        if "disabled" in clases.lower() or aria_disabled == "true":
            return False
        return True
    except:
        return False


# ============================================================================
# CONFIGURACIÓN DE CARPETAS
# ============================================================================

# 📌 Configura aquí tu carpeta de descargas base
base_download_path = os.getenv("DOWNLOAD_PATH")

# 📌 Crear carpeta sciencedirect dentro de descargas
sciencedirect_folder = os.path.join(base_download_path, "sciencedirect")
if not os.path.exists(sciencedirect_folder):
    os.makedirs(sciencedirect_folder)
    print(f"✅ Carpeta creada: {sciencedirect_folder}")
else:
    print(f"📁 Usando carpeta existente: {sciencedirect_folder}")

# Usar la carpeta sciencedirect como directorio de trabajo
download_path = sciencedirect_folder

# ============================================================================
# CÓDIGO PRINCIPAL SIN LIMITADOR DE PÁGINAS
# ============================================================================

# Detectar automáticamente el total de páginas
print("🔍 Detectando número total de páginas...")
total_pages = obtener_total_paginas(driver)

# Si no se pudo detectar, usar detección dinámica
if total_pages is None:
    print("📋 Modo dinámico activado: se procesarán todas las páginas disponibles")
    usar_deteccion_dinamica = True
    max_pages = float('inf')  # Sin límite
else:
    print(f"📊 Se procesarán {total_pages} páginas en total")
    usar_deteccion_dinamica = False
    max_pages = total_pages

page_numScience = 1
paginas_procesadas = 0

while True:
    print(f"\n{'='*50}")
    if usar_deteccion_dinamica:
        print(f"Procesando página {page_numScience}...")
    else:
        print(f"Procesando página {page_numScience} de {max_pages}...")

    try:
        # Seleccionar checkbox real (input, no span)
        checkbox_input = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#srp-toolbar input[type='checkbox']"))
        )

        # Marcarlo si no está marcado
        if not checkbox_input.is_selected():
            driver.execute_script("arguments[0].click();", checkbox_input)
            time.sleep(1)

        # Confirmar que quedó marcado
        if checkbox_input.is_selected():
            print("✅ Checkbox de 'todos los artículos' marcado correctamente")
        else:
            print("⚠️ No se pudo marcar el checkbox, reintentando...")
            driver.execute_script("arguments[0].click();", checkbox_input)
            time.sleep(1)

        # Botón de exportar
        export_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "span.export-all-link-text"))
        )
        driver.execute_script("arguments[0].click();", export_button)
        time.sleep(1)

        # Botón BibTeX
        bibtex_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-aa-button='srp-export-multi-bibtex']"))
        )

        intentos_descarga = 0
        descarga_exitosa = False
        
        # Guardar estado de archivos antes de la descarga
        archivos_antes_base = set([f for f in os.listdir(base_download_path) if f.endswith(".bib")])
        
        while intentos_descarga < 3 and not descarga_exitosa:
            try:
                bibtex_button.click()
                print(f"Intento de descarga #{intentos_descarga+1} para página {page_numScience}")
                
                # Esperar el archivo en la carpeta base de descargas
                end_time = time.time() + 20
                nuevo_archivo = None
                while time.time() < end_time:
                    archivos_actuales = set(f for f in os.listdir(base_download_path) if f.endswith(".bib"))
                    nuevos = archivos_actuales - archivos_antes_base
                    if nuevos:
                        archivo_descargado = max([os.path.join(base_download_path, f) for f in nuevos],
                                          key=os.path.getctime)
                        # Mover a la carpeta sciencedirect
                        nuevo_nombre = f"sciencedirect_page_{page_numScience}.bib"
                        ruta_nueva = os.path.join(download_path, nuevo_nombre)
                        os.rename(archivo_descargado, ruta_nueva)
                        print(f"✅ Archivo guardado como: {nuevo_nombre}")
                        nuevo_archivo = ruta_nueva
                        descarga_exitosa = True
                        break
                    time.sleep(1)
                
                if not nuevo_archivo:
                    print("No se pudo detectar el archivo descargado")
                    
            except Exception as e:
                print(f"Error en el intento {intentos_descarga + 1}: {e}")
            
            intentos_descarga += 1
            if not descarga_exitosa and intentos_descarga < 3:
                time.sleep(2)

        if not descarga_exitosa:
            print(f"❌ No se pudo completar la descarga para la página {page_numScience}")

        # Cerrar menú exportar
        try:
            close_button = driver.find_element(By.CSS_SELECTOR, "button.export-close-button")
            close_button.click()
            time.sleep(1)
        except:
            pass

        paginas_procesadas += 1

        # Verificar si hay más páginas disponibles
        if usar_deteccion_dinamica:
            if not verificar_pagina_siguiente_existe(driver):
                print(f"🏁 No hay más páginas disponibles. Proceso completado.")
                break
        else:
            if page_numScience >= max_pages:
                print(f"🏁 Se han procesado todas las {max_pages} páginas.")
                break

        # Pasar a la siguiente página
        try:
            time.sleep(2)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            botonSiguientePag = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.XPATH, "//a[@data-aa-name='srp-next-page']"))
            )
            driver.execute_script("arguments[0].click();", botonSiguientePag)
            print(f"🔄 Avanzando a la página {page_numScience + 1}...")
            time.sleep(5)
            page_numScience += 1
        except Exception as e:
            print(f"⚠️ No se pudo avanzar de página: {e}")
            print("🏁 Posiblemente se han procesado todas las páginas disponibles.")
            break

    except Exception as e:
        print(f"⚠️ Error en la página {page_numScience}: {e}")
        # Intentar continuar con la siguiente página
        page_numScience += 1
        continue

print(f"\n🎉 ¡Proceso completado! Se procesaron {paginas_procesadas} páginas en total.")
print(f"📁 Archivos guardados en: {download_path}")

📁 Usando carpeta existente: /home/yep/Documentos/proyectoAnalisisAlgoritmos/descargas/sciencedirect
🔍 Detectando número total de páginas...
⚠️ No se pudo detectar el total de páginas automáticamente: Message: 
Stacktrace:
#0 0x561940e54fea <unknown>
#1 0x5619408d2d06 <unknown>
#2 0x561940924e6d <unknown>
#3 0x561940925101 <unknown>
#4 0x5619409736b4 <unknown>
#5 0x56194094ae3d <unknown>
#6 0x561940970b3b <unknown>
#7 0x56194094abe3 <unknown>
#8 0x5619409172d2 <unknown>
#9 0x561940917f91 <unknown>
#10 0x561940e189e8 <unknown>
#11 0x561940e1c84f <unknown>
#12 0x561940dffec9 <unknown>
#13 0x561940e1d3f5 <unknown>
#14 0x561940de574f <unknown>
#15 0x561940e41cc8 <unknown>
#16 0x561940e41ea3 <unknown>
#17 0x561940e53f83 <unknown>
#18 0x7f21183b4f54 start_thread
#19 0x7f211843832c __clone3

🔄 Usando detección dinámica página por página...
📋 Modo dinámico activado: se procesarán todas las páginas disponibles

Procesando página 1...
✅ Checkbox de 'todos los artículos' marcado correctamente
Inte

KeyboardInterrupt: 

## Paso 23 — Cerrar el navegador

Este bloque cierra la instancia del navegador y libera los recursos utilizados.

In [ ]:
# Cerrar el navegador de forma segura
if driver:
    try:
        print("Cerrando navegador...")
        driver.quit()
        print("Navegador cerrado correctamente")
    except Exception as e:
        print(f"ADVERTENCIA: Error al cerrar navegador: {e}")
else:
    print("INFO: No hay navegador activo para cerrar")